# Генератор датасета JD_Analysis

Проект [JD_Analysis](https://github.com/ZhongTr0n/JD_Analysis) содержит направленные ациклические графы, сохранённые в формате JSON, которые представляют собой древо информационных технологий, с перечислением их названий (вершины/nodes), связей (рёбра/edges) между ними и силой этих связей (веса/weights).

Данный проект содержит два графа:

| Название        | Вершин | Рёбер |
|-----------------|--------|-------|
| `jd_data.json`  | 249    | 668   |
| `jd_data2.json` | 177    | 415   |

Указанные графы имеют следующую структуру: 

```json
{
  "nodes": [
    {"id": "node1", "color": 1.23, "group": 1},
    {"id": "node2", "color": 1.23, "group": 2},
    {"id": "node3", "color": 1.23, "group": 2},
    {"id": "node4", "color": 1.23, "group": 3}
  ],
  "links": [
    {"source": "node1", "target": "node2", "value": 1.2, "group":1},
    {"source": "node2", "target": "node3", "value": 3.1, "group":2},
    {"source": "node3", "target": "node4", "value": 2.5, "group":3}
  ]
}
```

В блоке `nodes` перечислен список вершин графа, каждый элемент указанного списка содержит поля
- `id` которое содержит название технологии;
- `color` для обозначения цвета узла (так как автор проекта JD_Analysis использовал собранные данные для визуализации взаимосвязей между технологиями);
- `group`, которое скорее всего должно было представлять профессии к которым данные технологии относятся, но судя по тому, что на обоих графах в поле group всегда содержится число 0 можно предположить, что автор не стал развивать эту идею.

В блоке `links` перечислен список рёбер графа, каждое ребро представлено несколькими полями, в частности:
- `source` описывает из какой вершины начинается ребро;
- `target` описывает в какую вершину данное ребро приходит;
- `value` вес ребра;
- `group`, которое, как и в случае с nodes, тоже всегда содержит 0 и не представляет ценности. 

В данном юпитер-блокноте из указанных JSON файлов мы соберём датасет, который в дальнейшем можно будет загрузить на площадку HuggingFace и использовать в дальнейшем при помощи стандартизированных загрузчиков данных.

## Загрузка JSON файлов

Данный блок позволяет скачать указанные выше JSON файлы, после чего сохранит их в директории, в которой находится текущий юпитер-блокнот, после чего прочитать содержимое и добавить в список объектов.

In [6]:
from simple_gnn.dataset.jd_dataset import load_jda

JD_ANALYSIS_GRAPHS = ["jd_data.json", "jd_data2.json"]

items = {}
for jd in JD_ANALYSIS_GRAPHS:
    items[jd] = {'object': load_jda(jd)}

items

{'jd_data.json': {'object': {'nodes': [{'id': '.net',
     'color': 60.32785914264073,
     'group': 0},
    {'id': 'actionscript', 'color': 15.307012281730257, 'group': 0},
    {'id': 'adobe', 'color': 17.150977832309312, 'group': 0},
    {'id': 'agile', 'color': 52.356489380272656, 'group': 0},
    {'id': 'aix', 'color': 15.228536030355718, 'group': 0},
    {'id': 'ajax', 'color': 28.53085707112983, 'group': 0},
    {'id': 'algo', 'color': 15.023140579703956, 'group': 0},
    {'id': 'altova', 'color': 15.001870433034734, 'group': 0},
    {'id': 'amazon', 'color': 15.37350727198416, 'group': 0},
    {'id': 'android', 'color': 20.664544189867314, 'group': 0},
    {'id': 'ant', 'color': 15.322450548102168, 'group': 0},
    {'id': 'antlr', 'color': 15.000230050655402, 'group': 0},
    {'id': 'apache', 'color': 21.549456823572623, 'group': 0},
    {'id': 'apex', 'color': 15.084019073487719, 'group': 0},
    {'id': 'apl', 'color': 15.000230050655402, 'group': 0},
    {'id': 'app', 'color':

## Структура датасета

На данном этапе необходимо определиться со структурой будущего датасета.

В качестве примера был выбран формат предложенный сотрудниками HuggingFace в рамках проекта [Graph Datasets](https://huggingface.co/graphs-datasets), цель данного проекта - собирают разнообразную информацию, представленную в виде графов, после чего преобразовывать её и выполнять упаковку в формат датасета.

Чтобы создать датасет описывающий граф технологии потребуются поля:

- `node_feat` - список всех вершин графа, для каждой из них будет сгенерировано уникальное свойство (feature);
- `edge_index` - список всех рёбер графа;
- `edge_attr` - атрибуты рёбер, в данном случае вес;
- `y` - уникальные метки рёбер, по которым их можно будет в дальнейшем идентифицировать;
- `num_nodes` - количество вершин графа;
- `num_edges` - количество рёбер графа.

## Подготовка списка вершин

Сначала необходимо выполнить процедуру токенизации вершин.

Каждый токен будет представлять из себя пару число<->строка в которой число обозначает порядковый номер вершины в списке `nodes`, а строка будет описывать название вершины (название технологии).

В дальнейшем это позволит описать список `node_index` и использовать его на этапе генерации признаков узлов.

In [7]:
for jd in JD_ANALYSIS_GRAPHS:
    graph_data = items[jd]['object']

    # Получаем список вершин преобразовав словарь в список
    items[jd]['node_list'] = [node['id'] for node in graph_data['nodes']]

    # Получим словарь вида id->name
    items[jd]['node_index'] = {index: node for index, node in enumerate(items[jd]['node_list'])}

    # Получим обратный словарь вида name->id
    items[jd]['node_mapping'] = {node_id: i for i, node_id in enumerate(items[jd]['node_list'])}

items['jd_data.json']['node_index']

{0: '.net',
 1: 'actionscript',
 2: 'adobe',
 3: 'agile',
 4: 'aix',
 5: 'ajax',
 6: 'algo',
 7: 'altova',
 8: 'amazon',
 9: 'android',
 10: 'ant',
 11: 'antlr',
 12: 'apache',
 13: 'apex',
 14: 'apl',
 15: 'app',
 16: 'asp.net',
 17: 'asterdata',
 18: 'atom',
 19: 'awk',
 20: 'aws',
 21: 'backbone',
 22: 'bash',
 23: 'bdd',
 24: 'bigtable',
 25: 'blackberry',
 26: 'bluetooth',
 27: 'boost',
 28: 'c',
 29: 'cache',
 30: 'cake',
 31: 'cassandra',
 32: 'celery',
 33: 'cellular',
 34: 'centos',
 35: 'cfengine',
 36: 'cisco',
 37: 'clojure',
 38: 'cloud',
 39: 'cmake',
 40: 'cms',
 41: 'codeigniter',
 42: 'coffeescript',
 43: 'confluence',
 44: 'couch',
 45: 'couchdb',
 46: 'css',
 47: 'cucumber',
 48: 'cython',
 49: 'database',
 50: 'dell',
 51: 'dhcp',
 52: 'dhtml',
 53: 'directx',
 54: 'django',
 55: 'dns',
 56: 'dom',
 57: 'dreamweaver',
 58: 'drupal',
 59: 'eclipse',
 60: 'ejb',
 61: 'encryption',
 62: 'erlang',
 63: 'esx',
 64: 'exim',
 65: 'extjs',
 66: 'fcoe',
 67: 'flash',
 68: 'f

Теперь необходимо преобразовать идентификаторы узлов бинарный формат, поскольку количество вершин в обоих графах не превышает 256 преобразуем идентификаторы в 8-битные последовательности нулей и единиц, кодирующие каждое число.

In [8]:
import torch

for jd in JD_ANALYSIS_GRAPHS:
    node_list = items[jd]['node_list']

    # Получим для каждой вершины её параметры
    items[jd]['node_feat'] = torch.randn(len(node_list), 1)

items['jd_data.json']['node_feat']

tensor([[-0.1946],
        [ 0.9171],
        [ 0.0146],
        [ 0.2631],
        [-0.4793],
        [-1.1027],
        [-0.0346],
        [ 0.4621],
        [ 0.6394],
        [-0.0962],
        [-0.4980],
        [ 0.4156],
        [ 0.4514],
        [-0.5381],
        [ 1.7009],
        [ 0.2145],
        [ 0.9281],
        [ 0.9653],
        [ 0.5640],
        [ 2.2022],
        [-1.9457],
        [ 1.4011],
        [-1.1871],
        [ 0.3797],
        [-0.2612],
        [-0.5488],
        [ 2.5723],
        [ 0.9996],
        [-1.2492],
        [ 0.2011],
        [-0.9874],
        [-0.1073],
        [-0.7363],
        [-0.6878],
        [ 0.4067],
        [ 0.1815],
        [-0.0785],
        [-0.2589],
        [-1.1825],
        [ 0.2464],
        [-0.4852],
        [-0.3439],
        [-0.0104],
        [ 0.0256],
        [ 0.3717],
        [ 0.2502],
        [-1.0305],
        [-0.1110],
        [ 0.2756],
        [ 0.8182],
        [ 0.0763],
        [ 0.1885],
        [ 1.

## Подготовка рёбер

На данном этапе у нас уже имеется список вершин и различные словари для работы с ними, поэтому приступим к этапу преобразования списка `links` в формат связей числовых идентификаторов. Помимо этого потребуется извлечь веса рёбер из поля `value`.

In [9]:
for jd in JD_ANALYSIS_GRAPHS:
    graph_data = items[jd]['object']
    node_mapping = items[jd]['node_mapping']

    items[jd]['edge_index'] = [[node_mapping[link['source']], node_mapping[link['target']]] for link in graph_data['links']]
    items[jd]['edge_attr'] = [link['value'] for link in graph_data['links']]

items['jd_data.json']['edge_index'], items['jd_data.json']['edge_attr']

([[0, 3],
  [0, 5],
  [0, 16],
  [0, 28],
  [0, 46],
  [0, 89],
  [0, 102],
  [0, 111],
  [0, 131],
  [0, 136],
  [0, 142],
  [0, 146],
  [0, 184],
  [0, 189],
  [0, 190],
  [0, 192],
  [0, 199],
  [0, 216],
  [0, 230],
  [0, 231],
  [0, 238],
  [0, 243],
  [0, 244],
  [1, 67],
  [1, 156],
  [2, 46],
  [2, 57],
  [2, 67],
  [2, 156],
  [3, 16],
  [3, 23],
  [3, 28],
  [3, 101],
  [3, 119],
  [3, 136],
  [3, 142],
  [3, 146],
  [3, 148],
  [3, 184],
  [3, 187],
  [3, 198],
  [3, 216],
  [3, 238],
  [3, 243],
  [4, 93],
  [4, 123],
  [4, 175],
  [4, 196],
  [4, 227],
  [5, 16],
  [5, 46],
  [5, 89],
  [5, 102],
  [5, 111],
  [5, 113],
  [5, 114],
  [5, 136],
  [5, 244],
  [6, 37],
  [6, 72],
  [6, 85],
  [6, 116],
  [6, 183],
  [8, 20],
  [8, 38],
  [9, 15],
  [9, 25],
  [9, 95],
  [9, 101],
  [9, 147],
  [10, 59],
  [10, 101],
  [10, 117],
  [10, 129],
  [10, 218],
  [12, 34],
  [12, 103],
  [12, 122],
  [12, 123],
  [12, 137],
  [12, 141],
  [12, 155],
  [12, 157],
  [12, 218],
  [12, 

## Создание таблицы больших графов

Поскольку все необходимые данные уже имеются можно упаковать их в формат Apache PyArrow Parquet, используемый для хранения датасетов на HuggingFace.

In [10]:
import pyarrow as pa

# Ключи которые нас интересуют
keys = ['node_feat', 'node_index', 'edge_index', 'edge_attr', 'node_list']

# Инициализируем объект списков
t_data = {key: [] for key in keys + ['num_nodes', 'num_edges']}

# Пройдёмся по каждому подготовленному дотасету
for jd in JD_ANALYSIS_GRAPHS:
    for key in keys:
        if key == 'node_feat':
            t_data[key].append(items[jd][key].tolist())
        else:
            t_data[key].append(items[jd][key])

    # Посчитаем 'num_nodes' и 'num_edges'
    t_data['num_nodes'].append(len(items[jd]['node_index']))
    t_data['num_edges'].append(len(items[jd]['edge_index']))

# Соберём таблицу с двумя графами
large_table = pa.table(
    data={
        'name': JD_ANALYSIS_GRAPHS,
        'node_feat': t_data['node_feat'],
        'edge_index': t_data['edge_index'],
        'edge_attr': t_data['edge_attr'],
        'y': t_data['node_list'],
        'num_nodes': t_data['num_nodes'],
        'num_edges': t_data['num_edges'],
    },
    schema=pa.schema([
        pa.field(name="name", type=pa.string()),
        pa.field(name="node_feat", type=pa.list_(pa.list_(pa.float64()))),
        pa.field(name="edge_index", type=pa.list_(pa.list_(pa.int64()))),
        pa.field(name="edge_attr", type=pa.list_(pa.float64())),
        pa.field(name="y", type=pa.list_(pa.string())),
        pa.field(name="num_nodes", type=pa.int64()),
        pa.field(name="num_edges", type=pa.int64()),
    ]),
)

large_table

pyarrow.Table
name: string
node_feat: list<item: list<item: double>>
  child 0, item: list<item: double>
      child 0, item: double
edge_index: list<item: list<item: int64>>
  child 0, item: list<item: int64>
      child 0, item: int64
edge_attr: list<item: double>
  child 0, item: double
y: list<item: string>
  child 0, item: string
num_nodes: int64
num_edges: int64
----
name: [["jd_data.json","jd_data2.json"]]
node_feat: [[[[-0.1946171522140503],[0.9170738458633423],...,[-0.31372490525245667],[0.10580454766750336]],[[0.9915452599525452],[-2.989278554916382],...,[0.4129026532173157],[1.4734593629837036]]]]
edge_index: [[[[0,3],[0,5],...,[230,231],[238,243]],[[0,12],[0,92],...,[173,174],[174,175]]]]
edge_attr: [[[0.8372881714433367,0.44773339967812265,2.972593634658615,2.8562696820640907,1.122948220895545,...,0.3043251194585422,1.9172178562464413,1.8420892465857406,3,2.676464392590603],[1.2988102697554484,0.9963664180718397,0.7956228463571555,0.6893503066233211,1.775953923744971,...,0

## Создание таблицы маленьких графов

Сначала сгенерируем 1000 подграфов из каждого большого графа, после чего соберём их в таблицу.

In [11]:
from simple_gnn.graph.generate_subgraphs import *
from simple_gnn.dataset.jd_dataset import *

# Сгенерируем 1000 подграфов из каждого большого графа
for jd in JD_ANALYSIS_GRAPHS:
    keys = ['name', 'node_feat', 'node_index', 'edge_index', 'edge_attr', 'node_list']
    t_data = {key: [] for key in keys + ['num_nodes', 'num_edges']}
    graph_data = items[jd]['object']
    subgraphs = generate_subgraphs(graph_data, 1000, 2, 15)
    for (index, subgraph) in enumerate(subgraphs):

        # Build edge index
        subgraph_edge_index = []
        for link in subgraph['links']:
            source_idx = items[jd]['node_mapping'].get(link['source'])
            target_idx = items[jd]['node_mapping'].get(link['target'])
            # Add edge only if both nodes are on the subgraph
            if source_idx is not None and target_idx is not None:
                subgraph_edge_index.append([source_idx, target_idx])

        # Build edges weights
        subgraph_edge_attr = [link['value'] for link in subgraph['links']]

        # List of nodes 
        subgraph_node_list = [link['id'] for link in subgraph['nodes']]

        # Convert subgraphs nodes of the small graph
        subgraph_node_index = []
        for link in subgraph['nodes']:
            node_idx = items[jd]['node_mapping'].get(link['id'])
            if node_idx is not None:
                subgraph_node_index.append(node_idx)

        # Make a mask for the subgraph nodes
        subgraph_mask = torch.zeros_like(items[jd]['node_feat'])
        for idx in subgraph_node_index:
            subgraph_mask[idx] = 1
        subgraph_masked_features = items[jd]['node_feat'] * subgraph_mask

        t_data['name'].append(jd)
        t_data['node_feat'].append(subgraph_masked_features.tolist())
        t_data['node_index'].append(subgraph_node_index)
        t_data['node_list'].append(subgraph_node_list)
        t_data['edge_index'].append(subgraph_edge_index)
        t_data['edge_attr'].append(subgraph_edge_attr)
        t_data['num_nodes'].append(len(subgraph_node_index))
        t_data['num_edges'].append(len(subgraph_edge_index))

        items[jd]['subgraph_table'] = pa.table(
            data={
                'name': t_data['name'],
                'node_feat': t_data['node_feat'],
                'edge_index': t_data['edge_index'],
                'edge_attr': t_data['edge_attr'],
                'y': t_data['node_list'],
                'num_nodes': t_data['num_nodes'],
                'num_edges': t_data['num_edges'],
            },
            schema=pa.schema([
                pa.field(name="name", type=pa.string()),
                pa.field(name="node_feat", type=pa.list_(pa.list_(pa.float64()))),
                pa.field(name="edge_index", type=pa.list_(pa.list_(pa.int64()))),
                pa.field(name="edge_attr", type=pa.list_(pa.float64())),
                pa.field(name="y", type=pa.list_(pa.string())),
                pa.field(name="num_nodes", type=pa.int64()),
                pa.field(name="num_edges", type=pa.int64()),
            ]),
        )

In [12]:
from datasets import Dataset, DatasetDict

# Преобразование объектов PyArrow в объекты датасета Hugging Face
ds_large = Dataset(large_table)
ds_jd_data = Dataset(items["jd_data.json"]['subgraph_table'])
ds_jd_data2 = Dataset(items["jd_data2.json"]['subgraph_table'])

# Объединение отдельных датасетов в один датасет с разными сплитами
dataset_dict = DatasetDict({
    'train': ds_large,
    'jd_data.json': ds_jd_data,
    'jd_data2.json': ds_jd_data2
})

# Выгрузка датасета на Hugging Face
dataset_dict.push_to_hub('evilfreelancer/jd_analysis_graphs')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/791 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/evilfreelancer/jd_analysis_graphs/commit/e06107e4a7927066b6a9f3a26c77d8c3867b4e59', commit_message='Upload dataset', commit_description='', oid='e06107e4a7927066b6a9f3a26c77d8c3867b4e59', pr_url=None, pr_revision=None, pr_num=None)